In [40]:
import os
import numpy as np
import h5py
import json
import torch
from scipy.misc import imread, imresize
from tqdm import tqdm
from collections import Counter
from random import seed, choice, sample
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from random import randint

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /home/as3ek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [57]:
dataset = pd.read_csv('data/caption_data_0_100.csv') 
image_folder = 'data/images/' 
captions_per_image = 7000
min_word_freq = 5
output_folder = 'data/proc_data_files/'
max_len = 20

train_image_paths = []
train_image_captions = []
val_image_paths = []
val_image_captions = []
test_image_paths = []
test_image_captions = []
num_images_to_train = 20
word_freq = Counter()

In [58]:
for img in dataset['ImageName'][:num_images_to_train]:
    captions = []
    for c in dataset[dataset['ImageName'] == img]['Caption']:
        # Updating word freq
        c = str(c)
        
        tokens = word_tokenize(c)
        tokens = [token.lower() for token in tokens]
        
        word_freq.update(tokens)
        if len(tokens) <= max_len:
            captions.append(tokens)
    
    if len(captions) == 0:
        continue
    
    path = os.path.join(img)
    
    if randint(0, 10) < 9:
        train_image_paths.append(path)
        train_image_captions.append(captions)
    
    else:
        val_image_paths.append(path)
        val_image_captions.append(captions)
        
    if randint(0, 10) < 5:
        test_image_paths.append(path)
        test_image_captions.append(captions)
        
# Sanity check
assert len(train_image_paths) == len(train_image_captions)
assert len(val_image_paths) == len(val_image_captions)
assert len(test_image_paths) == len(test_image_captions)

In [59]:
# Create word map
words = [w for w in word_freq.keys() if word_freq[w] > min_word_freq]
word_map = {k: v + 1 for v, k in enumerate(words)}
word_map['<unk>'] = len(word_map) + 1
word_map['<start>'] = len(word_map) + 1
word_map['<end>'] = len(word_map) + 1
word_map['<pad>'] = 0

In [ ]:
# Create a base/root name for all output files
base_filename = dataset + '_' + str(captions_per_image) + '_cap_per_img_' + str(min_word_freq) + '_min_word_freq'